In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def draw_hlandmarks(frame,results):
    for num,hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(frame,hand,mp_hands.HAND_CONNECTIONS) #rendering
def extract_keypoints(results):
    if results.multi_hand_landmarks:
        return np.array([[lmark.x,lmark.y,lmark.z] for lmark in results.multi_hand_landmarks[0].landmark]).flatten() \
        if results.multi_hand_landmarks[0].landmark else np.zeros((21*3))
    else:
        return np.zeros((21*3))

In [4]:
DATA_PATH= os.path.join('Mediapipe_Data') #Path for exported data
actions = np.array(['Active','Draw','No Data Sent']) #Detect signs
no_sequences = 50 #number videos
sequences_length = 50 #number of frames

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [6]:
model = Sequential()
model.add(LSTM(64,return_sequences=True, activation='relu', input_shape=(50,63)))
model.add(LSTM(128,return_sequences=True, activation='relu'))
model.add(LSTM(64,return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [7]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [8]:
model.load_weights('gesture.h5')

In [10]:
#Testing in Real Time
import urx
import moveURX as URx
import moveRoboDK as roboDK
import collections
collections.Iterable = collections.abc.Iterable
rob = urx.Robot("192.168.0.112")
try:
    sequence = []
    result = "" 
    threshold = 0.90 #90% 
    vidcap = cv2.VideoCapture(0)
    with mp_hands.Hands(min_detection_confidence=0.8,min_tracking_confidence=0.5,max_num_hands=1) as hands:
        while vidcap.isOpened():
            localresult = ""
            ret,frame = vidcap.read()
            image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)#OpenCV works in BlueGreenRed format 
                                                         #but Mediapipe works in RedGreenBlue format
            image.flags.writeable = False #setting flags to prevent copying the image
            results = hands.process(image) #detection
            image.flags.writeable = True #setting it back
            image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #changing back
            #print(results) #output all the landmarks 
            if results.multi_hand_landmarks:
                draw_hlandmarks(frame,results) #rendering

            #predictions
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-50:]

            if len(sequence) == 50:
                res = model.predict(np.expand_dims(sequence,axis=0))[0] #get the result
                if res[np.argmax(res)] > threshold:
                    localresult = actions[np.argmax(res)]
                    if localresult != result and localresult != 'No Data Sent':
                        result = localresult
                        if result == 'Draw':
                            roboDK.moveRobottoPickUp_Reach('Draw')
                            URx.moveRobot('workspace',rob)
                        elif result == 'Active':
                            roboDK.moveRobottoPickUp_Reach('Active')
                            URx.moveRobot('pickup',rob)
            cv2.rectangle(frame,(0,0),(640,40),(245,117,16),-1)
            cv2.putText(frame,result,(3,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

            cv2.imshow('Hand Tracking',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        vidcap.release()
        cv2.destroyAllWindows()
finally:
    rob.close()

Flag value at beg: True
Flag value at end: False
Flag value at beg: True
Flag value at end: False
Flag value at beg: True
Flag value at end: False
Flag value at beg: True
Flag value at end: False
Flag value at beg: True
Flag value at end: False
Invalid item provided: The item identifier provided is not valid or it does not exist.


Exception: Invalid item provided: The item identifier provided is not valid or it does not exist.

In [11]:
vidcap.release()
cv2.destroyAllWindows()

NameError: name 'vidcap' is not defined